In [8]:
import { PromptTemplate } from "@langchain/core/prompts";

const greetingPrompt = new PromptTemplate({
  inputVariables: [],
  template: "hello world",
});
const formattedGreetingPrompt = await greetingPrompt.format();

console.log(formattedGreetingPrompt);


hello world


In [9]:
import { PromptTemplate } from "@langchain/core/prompts";

const greetingPrompt = new PromptTemplate({
  inputVariables: ["name"],
  template: "hello,{name}",
});
const formattedGreetingPrompt = await greetingPrompt.format({
    name: 'yyl'
});

console.log(formattedGreetingPrompt);


hello,yyl


In [10]:
const autoInterTemplate = PromptTemplate.fromTemplate("good {timeOfDay}, {name}")
console.log(autoInterTemplate.inputVariables);

const formattedAutoInferTemplate = await autoInterTemplate.format({
    timeOfDay: "morning",
    name: "yyl",
})
console.log(formattedAutoInferTemplate)

good morning, yyl


In [11]:
// 使用部分参数创建 template
const initialPrompt = new PromptTemplate({
    template: "this is a type of {type}, it is {item}",
    inputVariables: ['type', 'item'],
})

const partialedPrompt = await initialPrompt.partial({
    type: "flower"
})
const formattedPrompt = await partialedPrompt.format({
    item: "rose"
})
console.log(formattedPrompt)

const formattedPrompt2 = await partialedPrompt.format({
    item: "Peony"
})
console.log(formattedPrompt2)

this is a type of flower, it is Peony


In [12]:
// 使用动态填充参数
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString()
}

const promptWithDate = new PromptTemplate({
    template: "today is {date}, {activity}",
    inputVariables: ["date", "activity"]
})

const partialedPromptWithDate = await promptWithDate.partial({
    date: getCurrentDateStr,
})

const formattedPromptWithDate = await partialedPromptWithDate.format({
    activity: "let's go to cafe"
})
console.log(formattedPromptWithDate);

today is 8/25/2024, let's go to cafe


In [16]:
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString()
}

function generateGreeting(timeOfDay) {
    return () => {
        const date = getCurrentDateStr()
        switch(timeOfDay){
            case 'morning':
                return date + ' good morning';
            case 'afternoon':
                return date + ' good afternoon';
            case 'evening':
                return date + ' good evening';
            default: 
                return date + ' hello'
        }
    }
}
const prompt = new PromptTemplate({
    template: "{greeting}!",
    inputVariables: ["greeting"]
})
const currentTimeOfDay = 'afternoon'
const partialedPrompt = await prompt.partial({
    greeting: generateGreeting(currentTimeOfDay)
})
const formattedPrompt = await partialedPrompt.format();
console.log(formattedPrompt);


8/25/2024 good afternoon!


In [35]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts";

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(`你是一个专
业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。`);



import { HumanMessagePromptTemplate } from "@langchain/core/prompts";
const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate("请翻译这句话：{text}")

import { ChatPromptTemplate} from "@langchain/core/prompts";
const chatPrompt = ChatPromptTemplate.fromMessages([
    translateInstructionTemplate,
    userQuestionTemplate
])
const formattedChatPrompt = await chatPrompt.formatMessages({
    source_lang: "Chinese",
    target_lang: "English",
    text: '你好'
})
console.log(formattedChatPrompt);


[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你是一个专\n业的翻译员，你的任务是将文本从Chinese翻译成English。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你是一个专\n业的翻译员，你的任务是将文本从Chinese翻译成English。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "请翻译这句话：你好",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "请翻译这句话：你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


: 

In [1]:
// 组合多个 Prompt
import { PromptTemplate, PipelinePromptTemplate } from "@langchain/core/prompts"
const getCurrentDateStr = () => {
    return new Date().toLocaleDateString()
}
const fullPrompt = PromptTemplate.fromTemplate(`你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
根据上下文，完成主人的需求 {task}
`)

const datePrompt =  PromptTemplate.fromTemplate("{date}, 现在是 {period}")
const periodPrompt = await datePrompt.partial({
    date: getCurrentDateStr
})
const infoPrompt = PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}")
const taskPrompt = PromptTemplate.fromTemplate(`
    我想吃 {period} 的 {food}。 
    再重复一遍我的信息 {info}
`)
const composedPrompt = new PipelinePromptTemplate({
    pipelinePrompts: [
        {
            name: "date",
            prompt: periodPrompt,
        },
        {
            name: "info",
            prompt: infoPrompt,
        },
        {
            name: "task",
            prompt: taskPrompt
        }
    ],
    finalPrompt: fullPrompt,
})
const formattedPrompt = await composedPrompt.format({
    period: "morning",
    name: "yyl",
    gender: "male",
    food: "lemon"
})
console.log(formattedPrompt);

你是一个智能管家，今天是 8/25/2024, 现在是 morning，你的主人的信息是姓名是 yyl, 性别是 male, 
根据上下文，完成主人的需求 
    我想吃 morning 的 lemon。 
    再重复一遍我的信息 姓名是 yyl, 性别是 male


